In [2]:
import os

from groq import Groq
from dotenv import load_dotenv

load_dotenv() 

client = Groq(
    api_key=os.getenv("GROQ_API_KEY")
)

In [3]:
def get_key_phrase(text):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f'''
                from the following text, extract the keyphrases and write them down as an array, be careful to keep the phrases intact, do not edit them. Note that keyphrases are often longer than keywords and they can help learners understand the main idea of ​​the text by reading only the keyphrases. There may be no keyphrases if the text has no specific content:
                Write only an array containing keyphrases or an empty array. Do not write any additional information. Do not introduce. Just generate [...]
                text:
                {text}''',
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    return chat_completion.choices[0].message.content

In [4]:
import re

def extract_text_between_brackets(text):
    # Sử dụng regex để tìm các đoạn văn bản nằm giữa [ và ]
    result = re.findall(r'\[(.*?)\]', text)
    if not result:
        return '<NONE>'
    return result[0]

In [5]:
import re

def remove_puctuation(text):
    return re.sub(r'[^\w\s]', '', text)

In [6]:
import ast

def get_final_keyphrase(text):
    llm_out = get_key_phrase(text)
    keyphrases = extract_text_between_brackets(llm_out).split('", "')
    keyphrases = [s.strip('"') for s in keyphrases]
    for key in keyphrases:
        key = remove_puctuation(key)
    return keyphrases

In [7]:
def label_keyphrase(texts, keyphrases):
    # Khởi tạo mảng label với giá trị mặc định là 0, độ dài bằng số từ trong câu
    
    labels = [0] * len(texts)

    # Xử lý từng keyphrase trong mảng keyphrases
    for keyphrase in keyphrases:
        keyphrase_tokens = keyphrase.split()
        n = len(keyphrase_tokens)

        # Tìm vị trí bắt đầu của keyphrase trong câu
        for i in range(len(texts) - n + 1):
            if texts[i:i + n] == keyphrase_tokens:
                # Gán giá trị 1 cho các vị trí thuộc keyphrase
                for j in range(i, i + n):
                    labels[j] = 1

    return labels

In [9]:
def gen_comments():
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": '''
                Generate exactly 50 Vietnamese specific random comments on Social Media. Write only an array containing comments. Do not write any additional information.
                Do not write " "
                ''',
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    return chat_completion.choices[0].message.content


In [10]:
def gen_data():
    llm_out = gen_comments()
    arr = re.findall(r'\[(.*?)\]', llm_out, re.DOTALL)
    comments = arr[0].split('\n')
    return comments

In [36]:
comments = []

In [37]:
arr = gen_data()
for a in arr:
    comments.append(a)

In [38]:
comments

['Tốt quá, mình sẽ thử ngay,',
 'Em đã bao giờ đi Đà Nẵng chưa,',
 'Sài Gòn ơi, mình thích bạn quá,',
 'Mình đang tìm nơi đi chơi dịp cuối tuần, có gợi ý nào không,',
 'Xin chào mọi người, mới tham gia cộng đồng này,',
 'Tất cả chúng ta đều có thể làm được mọi thứ nếu cố gắng,',
 'Đi Phuket dịp Tết này có ai muốn đi cùng không,',
 'Cần bán laptop cũ 5tr, còn rất mới,',
 'Cảm ơn bạn đã chia sẻ bài viết hữu ích này,',
 'Các bạn đã thử món bánh xèo tôm thịt chưa,',
 'Bạn nào có cửa hàng quần áo trẻ em không,',
 'Cuộc sống dù có khó khăn nhưng chúng ta vẫn phải cố gắng,',
 'Người Việt Nam mình vốn dĩ đã rất giỏi,',
 'Tôi thích đọc sách về lịch sử,',
 'Sài Gòn làm mình nhớ mãi không quên,',
 'Bạn nào có chia sẻ kinh nghiệm học ngoại ngữ không,',
 'Tôi đang học tiếng Anh và cần người nói chuyện,',
 'Đi chơi Hà Nội vào dịp cuối tuần,',
 'Sách hay thì dù có dày cũng đáng đọc,',
 'Đây là lần đầu tiên mình đi du lịch nước ngoài,',
 'Văn hóa Việt Nam thật sự rất đa dạng,',
 'Bạn nào có chia sẻ về

In [40]:
for i in range(len(comments)):
    comments[i] = remove_puctuation(comments[i])

In [41]:
new_data = []

In [43]:
labels = []
texts = []
for i in range(len(comments)):
    text = comments[i].split()
    texts.append(text)
    keyphrases = get_final_keyphrase(comments[i])
    data = {
        "text":text,
        "customer_id": f'task-{i}',
        "explainatiom":"None",
        "keyphrase":keyphrases,
        "label": label_keyphrase(text, keyphrases)
    }
    new_data.append(data)

In [48]:
comments[3]

'Mình đang tìm nơi đi chơi dịp cuối tuần có gợi ý nào không'

In [46]:
new_data

[{'text': ['Bánh', 'flan', 'dừa', 'nướng', 'ngon', 'quá'],
  'customer_id': 'task-0',
  'explainatiom': 'None',
  'keyphrase': [''],
  'label': [0, 0, 0, 0, 0, 0]},
 {'text': ['Bánh', 'flan', 'dừa', 'nướng', 'ngon', 'quá'],
  'customer_id': 'task-1',
  'explainatiom': 'None',
  'keyphrase': [''],
  'label': [0, 0, 0, 0, 0, 0]},
 {'text': ['Bánh', 'flan', 'dừa', 'nướng', 'ngon', 'quá'],
  'customer_id': 'task-2',
  'explainatiom': 'None',
  'keyphrase': ['Sài Gòn ơi mình thích bạn quá'],
  'label': [0, 0, 0, 0, 0, 0]},
 {'text': ['Bánh', 'flan', 'dừa', 'nướng', 'ngon', 'quá'],
  'customer_id': 'task-3',
  'explainatiom': 'None',
  'keyphrase': [''],
  'label': [0, 0, 0, 0, 0, 0]},
 {'text': ['Bánh', 'flan', 'dừa', 'nướng', 'ngon', 'quá'],
  'customer_id': 'task-4',
  'explainatiom': 'None',
  'keyphrase': ['Xin chào mọi người mới tham gia cộng đồng này'],
  'label': [0, 0, 0, 0, 0, 0]},
 {'text': ['Bánh', 'flan', 'dừa', 'nướng', 'ngon', 'quá'],
  'customer_id': 'task-5',
  'explainatiom

In [51]:
s = {
        "text":"2",
        "customer_id": 'task-{i}',
        "explainatiom":"None",
        "keyphrase": '0',
        "label": '0'
    }

d.append(s)

In [52]:
d

[{'text': '1',
  'customer_id': 'task-{i}',
  'explainatiom': 'None',
  'keyphrase': '0',
  'label': '0'},
 {'text': '2',
  'customer_id': 'task-{i}',
  'explainatiom': 'None',
  'keyphrase': '0',
  'label': '0'}]